# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [1]:
# Use this cell to write your code for Task 1
import pandas as pd
import numpy as np


df = pd.read_csv("loyalty.csv")

mapper_fillna = {
    "spend": 0,
    "first_month": 0,
    "items_in_first_month": 0,
    "region": "Unknown",
    "loyalty_years": "0-1",
    "joining_month": "Unknown",
    "promotion": "No"
}

df.first_month = df.first_month.replace('.', np.nan)
df = df.fillna(mapper_fillna)
df.first_month = df.first_month.astype("float")
df.promotion = df.promotion.apply(lambda val: val.lower().title())
df.loyalty_years = df.loyalty_years.astype("category")
df.loyalty_years = df.loyalty_years.cat.reorder_categories(['0-1', '1-3', '3-5', '5-10', '10+'], ordered=True)


clean_data = df.copy()
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   customer_id           1246 non-null   int64   
 1   spend                 1246 non-null   float64 
 2   first_month           1246 non-null   float64 
 3   items_in_first_month  1246 non-null   int64   
 4   region                1246 non-null   object  
 5   loyalty_years         1246 non-null   category
 6   joining_month         1246 non-null   object  
 7   promotion             1246 non-null   object  
dtypes: category(1), float64(2), int64(2), object(3)
memory usage: 69.7+ KB


# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [2]:
# Use this cell to write your code for Task 2

df = pd.read_csv("loyalty.csv")

spend_by_years = df.groupby("loyalty_years")["spend"].agg(avg_spend = "mean", var_spend = "var").round(2)
spend_by_years = spend_by_years.reset_index()
spend_by_years

,loyalty_years,avg_spend,var_spend
0,0-1,110.56,9.30
1,1-3,129.31,9.65
2,10+,117.41,16.72
3,3-5,124.55,11.09
4,5-10,135.15,14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [3]:
# Use this cell to write your code for Task 3
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(["customer_id", "spend"], axis=1)
X_test = test.drop("customer_id", axis=1)

y = train.spend

X = pd.get_dummies(X, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

pipe = Pipeline([
    ("scaler", MaxAbsScaler()),
    ("lr", LinearRegression())
])

pipe.fit(X, y)
y_pred_lr = pipe.predict(X_test)

base_result = pd.DataFrame({"customer_id": test.customer_id.values, "spend": y_pred_lr})
base_result.head(3)

,customer_id,spend
0,5,140.699644
1,7,148.730919
2,16,140.810384


# Task 4

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [4]:
# Use this cell to write your code for Task 4
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(["customer_id", "spend"], axis=1)
X_test = test.drop("customer_id", axis=1)

y = train.spend

X = pd.get_dummies(X, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", MaxAbsScaler()),
    ("gbr", GradientBoostingRegressor(subsample=0.8, max_depth=1))
])

params = {
    "gbr__max_features": [0.18,0.2,0.22],
    "gbr__n_estimators": [300,400,500,600,700]
}

grid = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring="neg_mean_squared_error")

grid.fit(X, y)
best_model = grid.best_estimator_

y_pred_sgbr = best_model.predict(X_test)

compare_result = pd.DataFrame({"customer_id": test.customer_id.values, "spend": y_pred_sgbr})
compare_result.head(3)

,customer_id,spend
0,5,140.917592
1,7,149.223575
2,16,139.764346
